# 简单的 CLIP 模型

[CLIP](https://openai.com/index/clip/) 是一个多模态模型。它能将图像和文本映射到同一个向量空间中，由此可以产生诸多应用。比如，通过计算图片与文本的相似性，可以用近似最近邻 (ANN) 从相册中检索与给定 query 语义相近的图片。此外，CLIP 的 Vision Encoder 可以作为特征提取器使用，用于生成的图像 Embedding。如果在 Vision Encoder 后加一个 fc 层，并且冻住骨干网络仅对 fc 层做训练，通常可以得到一个效果不错的图像分类器。

In [1]:
# !python -m pip install --upgrade pip setuptools -i https://mirrors.aliyun.com/pypi/simple/
# !python -m pip install git+https://github.com/openai/CLIP.git -i https://mirrors.aliyun.com/pypi/simple/
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124
# !pip install accelerate
# !pip install -U flash-attn --no-build-isolation
# https://visualstudio.microsoft.com/visual-cpp-build-tools/

In [2]:
import logging
import torch
import clip

import utils

from PIL import Image
from transformers import CLIPProcessor, CLIPModel

In [3]:
MODEL_PATH = 'workspace'
DATA_PATH = 'data'

model_path = utils.gen_abspath(directory='./', rel_path=MODEL_PATH)
img_path = utils.gen_abspath(directory=DATA_PATH, rel_path='cat.JPG')

# 设置日志记录
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# 检查 GPU 是否可用
device = "cuda" if torch.cuda.is_available() else "cpu"
logger.info(f"Using device: {device}")

INFO:__main__:Using device: cuda


## 1. 使用 transformers 加载 CLIP

### 1）CLIP 模型介绍

为了训练 CLIP，OpenAI 收集了 4 亿对图文数据进行训练。训练目标是让图片的特征向量与对应文本的特征向量在向量空间中靠得更近。训练采用多模态对比学习的方法。在一个 batch 中，对于每张图片，它的目标是找到当前 batch 中与之最匹配的文本，最大化与匹配文本的相似度（正样本），并同时最小化与其他文本的相似度（负样本）。

CLIP 训练了两个独立的编码器：

- **图像编码器**：通常使用 ResNet 或 Vision Transformer (ViT)。
- **文本编码器**：基于 Transformer 结构。

OpenAI 尝试了多种编码器，得出一个很直觉的结论：模型的效果与参数量呈现正相关。基本上使用参数越大的编码器，效果就越好。

### 2）用 CLIP 计算图文相似性分数

用 transformers 库加载 openai/clip-vit-base-patch32。并用一张猫的图片与两句话进行对比：

- a photo of a cat
- a photo of a dog

使用 CLIP 模型，计算猫的图片与每句话的相似性分数，取分数最高的句子作为图片的分类标签。验证模型能否有效区分猫和狗。

> **Note:** 值得注意的是 a photo of {item} 是一种 Prompt Engineer 方法。除了前面这个，OpenAI 还用了很多其他标签。比如：
> 
> ```
> a bad photo of a {}.
> a photo of many {}.
> a sculpture of a {}.
> a photo of the hard to see {}.
> a low resolution photo of the {}.
> a rendering of a {}.
> graffiti of a {}.
> ```

In [4]:
model = CLIPModel.from_pretrained(
    "openai/clip-vit-base-patch32",
    device_map=device,
    torch_dtype=torch.float16,
    cache_dir=model_path,
    # attn_implementation="flash_attention_2",
)

processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32", cache_dir=model_path)

C:\Users\luoch\miniconda3\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [5]:
inputs = processor(text=["a photo of a cat", "a photo of a dog"],
                   images=Image.open(img_path),
                   return_tensors="pt",
                   padding=True).to(device)

inputs.keys()

dict_keys(['input_ids', 'attention_mask', 'pixel_values'])

In [6]:
with torch.no_grad():
    with torch.autocast(device):
        outputs = model(**inputs)

logits_per_image = outputs.logits_per_image  # this is the image-text similarity score
probs = logits_per_image.softmax(dim=1)  # we can take the softmax to get the label probabilities
probs

tensor([[0.9941, 0.0056]], device='cuda:0', dtype=torch.float16)

## 2. 使用 CLIP 库加载 ViT 视觉编码模型

使用 CLIP 库加载一个使用 Vision Transformer (ViT) 作为视觉编码器的 CLIP 模型。

将图、文分别转换为 `image_features` 和 `text_features` 两个 Embedding，再计算相似度。

In [7]:
# 下载模型
model, preprocess = clip.load("ViT-B/32", device=device, download_root=model_path)

In [8]:
def generate_image_embedding(image_path):

    # 加载示例图像并处理
    try:
        image = preprocess(Image.open(image_path).convert("RGB")).unsqueeze(0).to(device)
    except FileNotFoundError:
        logger.error(f"The image file {image_path} was not found.")
        raise
    except Exception as e:
        logger.error(f"An error occurred while opening the image: {e}")
        raise

    print(image.shape)

    # 使用模型生成图像的 Embedding
    with torch.amp.autocast(device_type=device):  # 使用混合精度推理
        with torch.no_grad():
            image_features = model.encode_image(image)
    
    # 将 Embedding 转换为标准化的向量
    image_features /= image_features.norm(dim=-1, keepdim=True)
    return image_features

def generate_text_embedding(text_list):

    # 将文本转化为 tokens
    text_tokens = clip.tokenize(text_list).to(device)

    print(text_tokens.shape)

    # 使用模型生成文本的 Embedding
    with torch.amp.autocast(device_type=device):  # 使用混合精度推理
        with torch.no_grad():
            text_features = model.encode_text(text_tokens)

    # 将 Embedding 转换为标准化的向量
    text_features /= text_features.norm(dim=-1, keepdim=True)
    return text_features

In [9]:
# 获取文本 Embedding
# text_features = generate_text_embedding(["A photo of a cat!", "A photo of a dog"])
text_features = generate_text_embedding(["一只猫", "一条狗"])
text_features.shape, text_features.device

torch.Size([2, 77])


(torch.Size([2, 512]), device(type='cuda', index=0))

In [10]:
# 获取图片 Embedding

image_features = generate_image_embedding(img_path)
image_features.shape, image_features.device

torch.Size([1, 3, 224, 224])


(torch.Size([1, 512]), device(type='cuda', index=0))

In [11]:
# 计算图像与文本的相似性分数
eventual_similarity = torch.matmul(image_features, text_features.T)
eventual_similarity.cpu().numpy()

array([[0.281 , 0.2598]], dtype=float16)

参考：

- [OpenAI CLIP 官网](https://openai.com/index/clip/)
- [Hugging Face CLIP](https://huggingface.co/docs/transformers/en/model_doc/clip)
- [GitHub CLIP](https://github.com/openai/CLIP)
- [arXiv: *Learning Transferable Visual Models From Natural Language Supervision*](https://arxiv.org/abs/2103.00020)

视频资源：

- [CLIP 论文逐段精读](https://www.bilibili.com/video/BV1SL4y1s7LQ)
- [CLIP 改进工作串讲（上）](https://www.bilibili.com/video/BV1FV4y1p7Lm)
- [CLIP 改进工作串讲（下）](https://www.bilibili.com/video/BV1gg411U7n4)